In [1]:
import os
import glob
import pandas as pd
def get_data(dn):
    data = {
        'news':[],
        'ans':[]
    }
    dirs = glob.glob(os.path.join(dn, '*'))
    for d in dirs:
        pat = os.path.join(d,'*.[tT][xX][tT]')
        fs = glob.glob(pat)
        for fp in fs:
            with open(fp,'r',encoding='utf8') as f:
                news = f.read()
            ans = os.path.split(d)[-1]
            data['news'].append(news)
            data['ans'].append(ans)
    df = pd.DataFrame(data, columns=['news','ans'])
    return df

In [2]:
train_df = get_data('chinese_news_trans/chinese_news_trans/')
test_df = get_data('chinese_news_test/chinese_news_test/')
test_df

,news,ans
0,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
1,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
2,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
3,三趟火車停開 乘客可全額退票 \n瀏覽次數：1180 \n 昨日，來自鐵路部門的消息說...,交通
4,日月光華 -- Traffic_Info精華區文章閱讀\n-----------------...,交通
...,...,...
96,最優秀選手無緣亞運會健美賽\n \n \n 健美在亞洲運動會上是“新生兒”——韓國釜...,體育
97,\n各國記者眼中的羽毛球世錦賽\n \n\n-----------------------...,體育
98,\n友好運動會第五天 東道主選手大顯神威\n2001年09月03日02:51:40 新華社 ...,體育
99,不靠技術比運氣\n\n 第二屆奧運會在巴黎舉行，同時這裡也正在舉行國際博覽會，東道主把一些...,體育


In [13]:
#unique value_counts
kind = train_df['ans'].unique()
# enumerate
# kind
# trans = {k:i for i, k in enumerate(kind)}
reverse_trans = {i:k for i, k in enumerate(kind)}
reverse_trans

{0: '交通',
 1: '政治',
 2: '教育',
 3: '環境',
 4: '經濟',
 5: '藝術',
 6: '計算機',
 7: '軍事',
 8: '醫藥',
 9: '體育'}

In [4]:
y_train = train_df['ans'].replace(trans)
y_test = test_df['ans'].replace(trans)

In [5]:
import os 
from urllib.request import urlretrieve
DICT_PATH = 'dict.txt.big'
if not os.path.exists(DICT_PATH):
    print('install big dict')
    url = 'https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big'
    urlretrieve(url, DICT_PATH)

In [6]:
import jieba
jieba.set_dictionary(DICT_PATH)
def newscut(news):
    s = " ".join(jieba.cut(news))
    return s.replace('\r','').replace('\n','')
x_train_raw = train_df['news'].apply(newscut)
x_test_raw = test_df['news'].apply(newscut)

Building prefix dict from D:\dict.txt.big ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.ub18b883c92529549e6ceff6bcd158ca1.cache
Loading model cost 1.145 seconds.
Prefix dict has been built successfully.


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(x_train_raw)
x_test = vec.transform(x_test_raw)
x_test

<101x76399 sparse matrix of type '<class 'numpy.int64'>'
	with 27152 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, y_test)

1.0

In [10]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
col = ["{}(預測)".format(k) for k in kind]
row = ["{}(答案)".format(k) for k in kind]
pd.DataFrame(mat, columns = col, index=row)

,交通(預測),政治(預測),教育(預測),環境(預測),經濟(預測),藝術(預測),計算機(預測),軍事(預測),醫藥(預測),體育(預測)
交通(答案),10,0,0,0,0,0,0,0,0,0
政治(答案),0,11,0,0,0,0,0,0,0,0
教育(答案),0,0,10,0,0,0,0,0,0,0
環境(答案),0,0,0,10,0,0,0,0,0,0
經濟(答案),0,0,0,0,10,0,0,0,0,0
藝術(答案),0,0,0,0,0,10,0,0,0,0
計算機(答案),0,0,0,0,0,0,10,0,0,0
軍事(答案),0,0,0,0,0,0,0,10,0,0
醫藥(答案),0,0,0,0,0,0,0,0,10,0
體育(答案),0,0,0,0,0,0,0,0,0,10


In [11]:
n = input("請輸入新聞:")
test = vec.transform([newscut(n)])
pren = clf.predict(test)[0]
print("應該是:", reverse_trans[pren])
proba = clf.predict_proba(test)[0]
proba = list(enumerate(proba))
for i, prob in sorted(proba, key=lambda x:x[1], reverse=True):
    print(reverse_trans[i], ":", prob * 100, "%")

請輸入新聞:珊瑚公主號7人返台，目前已有4人確診，其餘3名第一次採檢陰性者也將列為明知故犯。指揮中心指揮官陳時中今天稍早宣布，將對他們徵收集中檢疫、檢驗費用，估計每人約自付新台幣5萬1000元。  陳時中在昨天的記者會上表示，增加的五名境外移入就有四個是來自珊瑚公主號，這艘郵輪只有7名台人，於2月15日至美國、祕魯、玻利維亞、巴西及智利等地旅遊，並在3月5日從智利登上「珊瑚公主號」郵輪，該郵輪原定3月19日於阿根廷結束旅程，但由於阿根廷等多國封鎖邊境導致郵輪無法停泊，因此該郵輪在海上漂流多日，並於4月4日上午停靠美國佛羅里達州邁阿密港。  當時船上已有12名旅客及10名船員確診感染，其中3名旅客死亡。
應該是: 經濟
經濟 : 49.97246158170842 %
軍事 : 29.434814783263825 %
體育 : 20.187891606052688 %
政治 : 0.40334339035142197 %
交通 : 0.001488624407812357 %
環境 : 1.1566604108688445e-08 %
醫藥 : 2.649037990649154e-09 %
藝術 : 4.160484582394866e-12 %
計算機 : 8.741606699426528e-15 %
教育 : 1.7651301012179987e-17 %


In [12]:
# list(zip([1,2,3],
#         [4,5,6]))